# Hanoi Weather Data Collection

This notebook demonstrates how to collect historical weather data for Hanoi from the Visual Crossing Weather API. We'll gather 10 years of daily weather data with 33 features that will be used for temperature forecasting.

## Objectives
1. Set up API connection to Visual Crossing Weather API
2. Collect 10 years of daily weather data for Hanoi
3. Understand the 33 weather features available
4. Validate and explore the collected data
5. Save data for further processing

#### **Note:** 
- This notebook is just the pseudocode for the data collection, the limit of records depends on your plan.
-  With the Visual Crossing Weather API, users on a free plan have a limit of 1000 records per day. Beyond that, they are billed for additional records if they are on a metered plan.

## 1. Setup and Imports

In [1]:
# Import required libraries
import os
import urllib.request
import csv
import codecs

## 2. API Configuration

Visual Crossing Weather API provides comprehensive historical weather data. We'll configure our API connection and understand the available parameters.

In [2]:
# API Configuration
API_KEY = os.getenv('VISUAL_CROSSING_API_KEY')  
BASE_URL = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
LOCATION = "Hanoi,Vietnam"

if not API_KEY:
    print("⚠️ Warning: API key not found!")
    print("Please set VISUAL_CROSSING_API_KEY in your .env file")
    print("You can get a free API key from: https://www.visualcrossing.com/weather/weather-data-services")
    API_KEY = input("Or enter your API key here: ")

print(f"API configured for location: {LOCATION}")
print(f"Base URL: {BASE_URL}")

⚠️ Warning: API key not found!
Please set VISUAL_CROSSING_API_KEY in your .env file
You can get a free API key from: https://www.visualcrossing.com/weather/weather-data-services
API configured for location: Hanoi,Vietnam
Base URL: https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline


## 3. Collect 10 Years of Daily Weather Data

In [9]:
import urllib.request
import csv
import codecs

# Setting up request parameters
base_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
location = 'Hanoi'
start_date = '2024-10-01'
end_date = '2025-10-01'
api_key = API_KEY
unit_group = 'metric'
content_type = 'csv'
include = 'days'

# Constructing the API URL
url = f"{base_url}{location}/{start_date}/{end_date}?key={api_key}&unitGroup={unit_group}&contentType={content_type}&include={include}"

try:
    # Open the URL
    with urllib.request.urlopen(url) as response:
        reader = csv.reader(codecs.iterdecode(response, 'utf-8'))
        
        # Iterate and print each row of CSV
        for row in reader:
            print(row)
except urllib.error.HTTPError as e:
    error_message = e.read()
    print(f"HTTP error: {e.code}, {error_message}")
except urllib.error.URLError as e:
    print(f"URL error: {e.reason}")

['name', 'datetime', 'tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'preciptype', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'sunrise', 'sunset', 'moonphase', 'conditions', 'description', 'icon', 'stations']
['Hanoi', '2024-10-01', '32.7', '24.9', '27.7', '41.6', '24.9', '30.9', '23.4', '78.6', '5.926', '100', '12.5', 'rain', '0', '0', '42.5', '27.7', '19.6', '1009.3', '79.7', '9.7', '130', '11.2', '6', '30', '2024-10-01T05:47:44', '2024-10-01T17:44:30', '0.95', 'Rain, Partially cloudy', 'Partly cloudy throughout the day with rain.', 'rain', '48820099999,48823099999,48825099999,48831099999,VVNB']
['Hanoi', '2024-10-02', '29.2', '23.7', '25.9', '30.1', '23.7', '26.1', '17.9', '62.1', '0.098', '100', '4.17', 'rain', '0', '0', '36.4', '20.5', '2.1', '1015.1', '47.2', '10.4', '235.3', '2